# Executa SQL Query para todos os valores numa coluna #

Imagine-se uma tabela como a seguinte:

| ID_PERCURSO | PARAGEM |    DIA     | GEOM |
| ----------- | ------- | ---------- | ---- |
|      0      |   255   |'2018-01-01 | xxxx |
|      0      |   255   |'2018-01-01 | xxxx |
|      0      |   254   |'2018-01-01 | xxxx |
|      0      |   254   |'2018-01-01 | xxxx |
|      0      |   255   |'2018-01-02 | xxxx |
|      0      |   255   |'2018-01-02 | xxxx |
|      0      |   254   |'2018-01-02 | xxxx |
|      0      |   254   |'2018-01-02 | xxxx |

E a seguinte Query:

```SQL
SELECT id_percurso, paragem, geom, dia, COUNT(paragem) AS conta
FROM TABLE
WHERE dia={}
GROUP BY id_percurso, paragem, geom, dia
```

Se pretendermos executar esta SQL Query para todos os dias em DIA e guardar o resultado numa nova tabela, este programa responde ao solicitado, executando, para cada dia em DIA, a QUERY substituindo "dia={}" por "dia='2018-01-01'" e depois por "dia='2018-01-02'".

In [ ]:
from glass.pgsql.anls import run_query_for_values_in_col

PGSQL_CONEXAO = {
    'HOST':'localhost', 'USER'    : 'postgres'   , 'PASSWORD': 'admin',
    'PORT': '5432'    , 'DATABASE': 'valcarris_v2'
}

QUERY = (
    "SELECT route_id, paragem, dia, COUNT(paragem) AS conta "
    "FROM validacoes_v9 "
    "WHERE route_id='736|0|ASC' AND dia={} "
    "GROUP BY route_id, paragem, dia"
)

TABELA_INTERESSE = "validacoes_v9"

COLUNA_INTERESSE = "dia"

PASTA_SAIDA = r'D:\TRENMO_JASP\tst_script\pgsql'

run_query_for_values_in_col(PGSQL_CONEXAO, QUERY, TABELA_INTERESSE, COLUNA_INTERESSE, PASTA_SAIDA)

# Conta o número de linhas de uma tabela por período #

Este programa conta o número de linhas de uma determinada tabela com dados referentes a vários períodos. Os períodos são definidos a partir da duração que estes devem ter. Por exemplo, se o utilizador definir períodos com duração de 01:00:00, o programa dividirá a duração de um dia em períodos com intervalo de 01:00:00, e, para cada um deles, são contadas as linhas cujo tempo associado se insere nesse intervalo.

In [ ]:
from glass.pgsql.anls.count import count_by_periods_with_certain_duration

PGSQL_CONEXAO = {
    'HOST':'localhost', 'USER'    : 'postgres'   , 'PASSWORD': 'admin',
    'PORT': '5432'    , 'DATABASE': 'valcarris_v2'
}

INTERVALO        = "01:00:00"
TABELA_INTERESSE = "validacoes_v9"
TIME_COLUNA      = "time"
TABELA_SAIDA     = r'D:\TRENMO_JASP\tst_script\pgsql\valida_periodo1h.xlsx'
FILTRO           = "route_id='736|0|ASC' AND dia='2018-01-18'" 

count_by_periods_with_certain_duration(
    PGSQL_CONEXAO, INTERVALO, TABELA_INTERESSE, TIME_COLUNA,
    TABELA_SAIDA, filterWhere=FILTRO
)

# Conta o número de linhas de uma tabela por período e entidade #

Considerando a Tabela I:

| route_id  | paragem | dia        | time     |
| --------- | ------- | ---------- | -------- |
| 736|0|ASC |  3812   | 2018-01-18 | 08:28:16 |
| 736|0|ASC |  5825   | 2018-01-18 | 09:05:17 |
| 736|0|ASC |  5705   | 2018-01-18 | 06:33:32 |
| 736|0|ASC |  3812   | 2018-01-18 | 07:11:16 |
| 736|0|ASC |  5519   | 2018-01-18 | 08:30:55 |
| ......... | ....... | .......... | ........ |
| 736|0|ASC |  5519   | 2018-01-18 | 07:20:17 |
| 736|0|ASC |  3812   | 2018-01-18 | 07:28:18 |
| 736|0|ASC |  5705   | 2018-01-18 | 09:28:16 |
| 736|0|ASC |  3812   | 2018-01-18 | 09:15:14 |

e assumindo como entidade o campo paragem e períodos com duração de 01:00:00, o resultado será equivalente à Tabela II:

| paragem | s00_00_e01_00 | s01_00_e02_00 | ... | s22_00_e23_00 | s23_00_e23_59 |
| ------- | ------------- | ------------- | --- | ------------- | ------------- |
|   3812  |       N       |       N       | ... |       N       |       N       |
|   5825  |       N       |       N       | ... |       N       |       N       |
|   5705  |       N       |       N       | ... |       N       |       N       |
|   5519  |       N       |       N       | ... |       N       |       N       |

Na qual N é igual ao número de validações por paragem naquele período (coluna).

In [ ]:
from glass.pgsql.anls.count import count_entity_periods_with_certain_duration as cepwcd

PGSQL_CONEXAO = {
    'HOST':'localhost', 'USER'    : 'postgres'   , 'PASSWORD': 'admin',
    'PORT': '5432'    , 'DATABASE': 'valcarris_v2'
}

INTERVALO        = "01:00:00"
TABELA_INTERESSE = "validacoes_v9"
TIME_COLUNA      = "time"
ENTIDADE_COLUNA  = "paragem"
TABELA_SAIDA     = r'D:\TRENMO_JASP\tst_script\pgsql\valida_paragem_periodo1h.xlsx'
FILTRO           = "route_id='736|0|ASC' AND dia='2018-01-18'"

cepwcd(
    PGSQL_CONEXAO, INTERVALO, TABELA_INTERESSE, TIME_COLUNA,
    ENTIDADE_COLUNA, TABELA_SAIDA, filterWhere=FILTRO
)

## Conta linhas por atributos e junta os dados a uma _Feature Class_ ##

Tendo em consideração uma _Feature Class_ (Tabela I) e uma tabela do PostgreSQL (Tabela II) como:

Tabela I:

| paragem | attr_1 | ... | attr_n |
| ------- | ------ | --- | ------ |
|  10000  | xxxxxx | ... | xxxxxx |
|  10001  | xxxxxx | ... | xxxxxx |
|  10002  | xxxxxx | ... | xxxxxx |
|  10003  | xxxxxx | ... | xxxxxx |
|  10004  | xxxxxx | ... | xxxxxx |
|  10005  | xxxxxx | ... | xxxxxx |

Tabela II:

| route_id  | paragem | dia        | time     |
| --------- | ------- | ---------- | -------- |
| 736|0|ASC |  10000  | 2018-01-18 | 08:28:16 |
| 737|0|ASC |  10001  | 2018-01-18 | 09:05:17 |
| 735|0|ASC |  10000  | 2018-01-18 | 06:33:32 |
| 736|0|ASC |  10004  | 2018-01-18 | 07:11:16 |
| 736|0|ASC |  10005  | 2018-01-18 | 08:30:55 |
| ......... | ....... | .......... | ........ |
| 737|0|ASC |  10001  | 2018-01-18 | 07:20:17 |
| 736|1|ASC |  10005  | 2018-01-18 | 07:28:18 |
| 735|0|ASC |  10004  | 2018-01-18 | 09:28:16 |
| 735|0|ASC |  10000  | 2018-01-18 | 09:15:14 |

O resultado será o equivalente ao seguinte:

| paragem | attr_1 | ... | attr_n | 736|0|ASC | 737|0|ASC | 735|0|ASC |
| ------- | ------ | --- | ------ | --------- | --------- | --------- |
|  10000  | xxxxxx | ... | xxxxxx |     N     |     N     |     N     |
|  10001  | xxxxxx | ... | xxxxxx |     N     |     N     |     N     |
|  10002  | xxxxxx | ... | xxxxxx |     N     |     N     |     N     |
|  10003  | xxxxxx | ... | xxxxxx |     N     |     N     |     N     |
|  10004  | xxxxxx | ... | xxxxxx |     N     |     N     |     N     |
|  10005  | xxxxxx | ... | xxxxxx |     N     |     N     |     N     |

Em que N é o número de linhas que têm aqueles valores nos campos paragem e route_id.

In [ ]:
from glass.pgsql.anls.count import count_rows_by_entity_and_shpJoin


PGSQL_CONEXAO = {
    'HOST':'localhost', 'USER'    : 'postgres'   , 'PASSWORD': 'admin',
    'PORT': '5432'    , 'DATABASE': 'valcarris_v2'
}
TABELA_INTERESSE    = "validacoes_v9"
ENTIDADE_COLUNA     = "paragem"
PIVOT_COLUNA        = "route_id"
SHP_TABLE           = r"D:\TRENMO_JASP\tst_script\pgsql\pnt_percursos.shp"
ENTIDADE_COLUNA_SHP = "PARAGEM"
SHP_SAIDA           = r'D:\TRENMO_JASP\tst_script\pgsql\stops_val_bylinha.shp'
FILTRO              = "dia='2018-01-18'"

count_rows_by_entity_and_shpJoin(
    PGSQL_CONEXAO, TABELA_INTERESSE, ENTIDADE_COLUNA,
    PIVOT_COLUNA, SHP_TABLE, ENTIDADE_COLUNA_SHP,
    SHP_SAIDA, WHERE=FILTRO
)